<a href="https://colab.research.google.com/github/LucasAlegre/minicurso-rl-eramia25/blob/main/minicurso_rl_eramia25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizado por Reforço: Como Ensinar Robôs a Maximizar Recompensas na Prática

Minicurso realizado no dia 12 de novembro durante o ERAMIA 2025 no Instituto de Informática da UFRGS.

Autor: Lucas N. Alegre


## Modelando Problemas com Gymnasium

Gymnasium é a versão mantida do OpenAI Gym pela Farama Foundation. É uma biblioteca Python para desenvolvimento e comparação de algoritmos de Reinforcement Learning.

### Instalação

```bash
pip install gymnasium
pip install gymnasium[all]  # Para ambientes adicionais
```

In [1]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

In [2]:
# @title Definindo um Agente Aleatório
class RandomAgent:
    def __init__(self, env):
        self.env = env
    def eval(self, obs):
        return self.env.action_space.sample()

In [3]:
## 1. Estrutura Básica de um Ambiente

# Criar ambiente
env = gym.make('FrozenLake-v1', render_mode='rgb_array')

# Resetar ambiente
observation, info = env.reset(seed=42)

agent = RandomAgent(env)

print(f"Observation: {observation}")
print(f"Info: {info}")

# Loop de interação
for _ in range(100):
    # Escolher ação (aleatória neste exemplo)
    action = agent.eval(observation)
    
    # Executar ação
    observation, reward, terminated, truncated, info = env.step(action)
    
    # Verificar se episódio terminou
    if terminated or truncated:
        print(f"Episódio terminou! Recompensa: {reward}")
        observation, info = env.reset()
        break

env.close()

Observation: 0
Info: {'prob': 1}
Episódio terminou! Recompensa: 0


In [8]:
## 2. Componentes Principais

env = gym.make('FrozenLake-v1', render_mode='rgb_array')

# Observation Space
print(f"Observation space: {env.observation_space}")
print(f"Número de estados: {env.observation_space.n}")

# Action Space
print(f"\nAction space: {env.action_space}")
print(f"Número de ações: {env.action_space.n}")
print(f"Ações: 0=Esquerda, 1=Baixo, 2=Direita, 3=Cima")

env.close()

Observation space: Discrete(16)
Número de estados: 16

Action space: Discrete(4)
Número de ações: 4
Ações: 0=Esquerda, 1=Baixo, 2=Direita, 3=Cima


In [7]:
animate_agent(agent, env)

Game over! Your agent lasted 13 steps.


/tmp/ipykernel_93885/2810858833.py:22: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  axes[1].set_ylim(0, max(returns) * 1.2)


NameError: name 'HTML' is not defined

### Recursos Adicionais

- 📚 Documentação oficial: https://gymnasium.farama.org/
- 🎮 Lista de ambientes: https://gymnasium.farama.org/environments/
- 💻 GitHub: https://github.com/Farama-Foundation/Gymnasium

## Avaliando um Agente


In [6]:
# @title Animando o agente
from matplotlib import animation

def animate_agent(agent, env, num_frames=100):
  s = env.reset()
  fig, axes = plt.subplots(1, 2, figsize=(10, 5))
  im = axes[0].imshow(env.render())
  frames = [env.render()]
  returns = [0]
  env_active = True
  for step in range(num_frames):
    a = agent.eval(s)
    s, r, terminated, truncated, info = env.step(a)
    done = terminated or truncated
    frames.append(env.render())
    returns.append(r + returns[-1])
    if env_active and done:
      env_active = False
      print(f'Game over! Your agent lasted {step} steps.')
  axes[1].set_title('Cumulative returns', fontsize=20)
  axes[1].set_xlim(0, num_frames)
  axes[1].set_ylim(0, max(returns) * 1.2)
  line, = axes[1].plot([], [], lw=2)

  def init():
    line.set_data([], [])
    im.set_data(frames[0])
    return [im]

  def animate(i):
    line.set_data(np.arange(i), returns[:i])
    im.set_data(frames[i])
    return [im]

  anim = animation.FuncAnimation(fig, animate, init_func=init, frames=num_frames,
                                 interval=50)
  plt.close()
  return HTML(anim.to_jshtml())

# Navegando no GridWorld com Q-Learning Tabular


## Deep Q-Networks
